In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from matplotlib.pyplot import figure, imshow, axis, close
from matplotlib.image import imread
%matplotlib inline
from annoy import AnnoyIndex
import torch.nn.functional as F

imsize=224
### resnet stuff
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dd = .2
model_ft = models.resnet50(pretrained=True)
#for param in model_ft.parameters():
#    param.requires_grad = False
#print(model_ft)
color_nodes = 8
gender_nodes =  2
region_nodes = 4
fighting_nodes= 3 
alignment_nodes = 9

class multi_output_model(torch.nn.Module):
    def __init__(self, model_core,dd):
        super(multi_output_model, self).__init__()
        
        self.resnet_model = model_core
        
        self.x1 =  nn.Linear(512,256)
        nn.init.xavier_normal_(self.x1.weight)
        
        self.bn1 = nn.BatchNorm1d(256,eps = 2e-1)
        self.x2 =  nn.Linear(256,256)
        nn.init.xavier_normal_(self.x2.weight)
        self.bn2 = nn.BatchNorm1d(256,eps = 2e-1)
        #self.x3 =  nn.Linear(64,32)
        #nn.init.xavier_normal_(self.x3.weight)
        #comp head 1
        
        
        #heads
        self.y1o = nn.Linear(256,gender_nodes)
        nn.init.xavier_normal_(self.y1o.weight)#
        self.y2o = nn.Linear(256,region_nodes)
        nn.init.xavier_normal_(self.y2o.weight)
        self.y3o = nn.Linear(256,fighting_nodes)
        nn.init.xavier_normal_(self.y3o.weight)
        self.y4o = nn.Linear(256,alignment_nodes)
        nn.init.xavier_normal_(self.y4o.weight)
        self.y5o = nn.Linear(256,color_nodes)
        nn.init.xavier_normal_(self.y5o.weight)
        
        
        self.d_out = nn.Dropout(dd)
    def forward(self, x):
       
        x1 = self.resnet_model(x)
        #x1 =  F.relu(self.x1(x1))
        #x1 =  F.relu(self.x2(x1))
        
        x1 =  self.bn1(F.relu(self.x1(x1)))
        x1 =  self.bn2(F.relu(self.x2(x1)))
        #x = F.relu(self.x2(x))
        #x1 = F.relu(self.x3(x))
        
        # heads
        y1o = F.softmax(self.y1o(x1),dim=1)
        y2o = F.softmax(self.y2o(x1),dim=1)
        y3o = F.softmax(self.y3o(x1),dim=1)
        y4o = F.softmax(self.y4o(x1),dim=1)
        y5o = torch.sigmoid(self.y5o(x1)) #should be sigmoid
        
        #y1o = self.y1o(x1)
        #y2o = self.y2o(x1)
        #y3o = self.y3o(x1)
        #y4o = self.y4o(x1)
        #y5o = self.y5o(x1) #should be sigmoid
        
        return y1o, y2o, y3o, y4o, y5o
    
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 512)

model1 = multi_output_model(model_ft,dd)
#print(model1)
#model1 = model1.to(device)
model1.load_state_dict(torch.load('F:/fgo_multi_label/models/super_basic/resnet50_super_basic_added256x2_bn.pth'))
model1.eval()


loader = transforms.Compose([transforms.Resize((imsize,imsize)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


def image_loader(image_name):
    """load image, returns cuda tensor"""
    #image_loader_1 = PIL.Image.fromarray(image_name)
    image_loader_1 = PIL.Image.open(image_name).convert("RGB")
    image_loader_1 = loader(image_loader_1).float()
    image_loader_1 = Variable(image_loader_1, requires_grad=True)
    image_loader_1 = image_loader_1.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    y_pred = model1(image_loader_1)
    gen = y_pred[0][0].detach().numpy().tolist()
    reg =y_pred[1][0].detach().numpy().tolist()
    fight = y_pred[2][0].detach().numpy().tolist()
    ali = y_pred[3][0].detach().numpy().tolist()
    color = y_pred[4][0].detach().numpy().tolist()
    feature_list = gen+reg+fight+ali+color
    
    return feature_list#.view(-1).detach().numpy())#.tolist()[0]#.cuda()  #assumes that you're using GPU

def showImagesHorizontally(annoy_dict, base_img, dir_, list_of_files,index):
    fig = figure(figsize=(160,90))
    number_of_files = len(list_of_files)
    list_of_files = [base_img] + list_of_files
    first_bool = 0 
    for i in range(number_of_files):
        if first_bool ==0:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(list_of_files[i])
            first_bool = 1
            
        else:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(dir_+annoy_dict[str(list_of_files[i])])
        
        #image = imread(dir_l+annoy_dict[str(list_of_files[i])])
        imshow(image,aspect='equal')
        axis('off')
    fig.savefig('D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/examples_v3_mo/{}.png'.format(index))
    fig.clf()
    close(fig)
    

    
annoy_index = AnnoyIndex(26*2,metric='manhattan')
annoy_index.load('annoy_euclidean_fgo_v4_multioutput.ann')


True

In [2]:
import pandas as pd
head_dir = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/crop_holder/'
full_dir = 'F:/fgo_multi_label/test/'

dat = pd.read_csv('mapping_with_annoy.csv')
dat_second = pd.read_csv('test_set_head_base_mapping.csv')
dat_second.head()


,Unnamed: 0,dir_loc,head_name,original_image
0,0,D:/object_detection/tf_model_dir/models/resear...,0defe15146155708d28195cff615d18b--ears-anime-a...,0defe15146155708d28195cff615d18b--ears-anime-a...
1,1,D:/object_detection/tf_model_dir/models/resear...,331453-Fate_Series-Saber_Lily-748x1008_0.png,331453-Fate_Series-Saber_Lily-748x1008.png
2,2,D:/object_detection/tf_model_dir/models/resear...,692882_0.jpg,692882.jpg
3,3,D:/object_detection/tf_model_dir/models/resear...,692882_1.jpg,692882.jpg
4,4,D:/object_detection/tf_model_dir/models/resear...,anime-fatestay-night-49204_0.jpg,anime-fatestay-night-49204.jpg


In [3]:

dict_fgo_annoy = {}
for index,row in dat.iterrows():
    dict_fgo_annoy[str(row['annoy_index'])] = str(row['original_image']) #switch this out

    

for index,row in dat_second.iterrows():
    head_name = row['head_name']
    img_name = row['original_image']
    feature_vec = image_loader(head_dir+head_name)
    feature_vec_full = image_loader(full_dir+img_name)
    
    feature_vec_final = feature_vec+feature_vec_full
    
    test_list = annoy_index.get_nns_by_vector(feature_vec_final, 5)
    
    base_image_path = full_dir+img_name
    dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/all_images/'

    showImagesHorizontally(dict_fgo_annoy,base_image_path,dir_loc,test_list,index=index)      

#dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/'

#for test_image in test_images:
    
#    img_name = test_image
#    feature_vec = image_loader(dir_loc+'heads/'+img_name)
#    test_list = annoy_index.get_nns_by_vector(feature_vec, 10)

    #dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/heads/'



#    showImagesHorizontally(dict_fgo_annoy,dir_loc,test_list)      